In [5]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix

GRAPH_SEED = 11
GRAPH_LEARN_MEAN = 0.5
GRAPH_LEARN_STD = 0.01
GRAPH_LEARN_EPOCHS = 5000
GRAPH_LEARN_ALPHA = 1.0
GRAPH_LEARN_BETA = 2.0
GRAPH_LEARN_ETA = 0.001

def learn_graph(X: torch.Tensor, seed_graph: int) -> np.ndarray:
    X_np = X.cpu().numpy()
    N = X_np.shape[0]
    rng = np.random.default_rng(seed_graph)
    W = rng.normal(GRAPH_LEARN_MEAN, GRAPH_LEARN_STD, size=(N, N))

    i_idx, j_idx = np.triu_indices(N, k=1)
    w = W[i_idx, j_idx].copy()
    delL = np.sum((X_np[i_idx] - X_np[j_idx]) ** 2, axis=1)

    row = np.concatenate([i_idx, j_idx])
    col = np.concatenate([np.arange(len(i_idx)), np.arange(len(j_idx))])
    S = coo_matrix((np.ones(len(row)), (row, col)), shape=(N, len(i_idx))).tocsc()
    S_T = S.T

    best_w = w.copy()
    best_loss = np.inf

    for _ in range(GRAPH_LEARN_EPOCHS):
        Sw = S.dot(w)
        inv_Sw = np.zeros_like(Sw)
        nonzero = Sw > 0
        inv_Sw[nonzero] = 1.0 / Sw[nonzero]
        grad = 2.0 * delL + GRAPH_LEARN_BETA * w - GRAPH_LEARN_ALPHA * S_T.dot(inv_Sw)
        w -= GRAPH_LEARN_ETA * grad
        np.clip(w, 0.0, None, out=w)

        loss = (delL * w).sum() + (GRAPH_LEARN_BETA / 2.0) * (w ** 2).sum() - GRAPH_LEARN_ALPHA * np.sum(np.log(Sw[nonzero]))
        if loss < best_loss:
            best_loss = loss
            best_w = w.copy()

    k = max(1, int(0.05 * len(best_w)))
    threshold = np.partition(best_w, -k)[-k]
    top_mask = best_w >= threshold

    adj = np.zeros((N, N), dtype=int)
    adj[i_idx[top_mask], j_idx[top_mask]] = 1
    adj[j_idx[top_mask], i_idx[top_mask]] = 1

    return adj

# ----- Graph Construction -----
df = pd.read_csv("10000_cylindrical_anomalies.csv")
X_raw = torch.tensor(df.iloc[:, :22].values, dtype=torch.float32)
scaler = StandardScaler()
X_scaled = torch.tensor(scaler.fit_transform(np.abs(X_raw.numpy())), dtype=torch.float32)

adj_matrix = learn_graph(X_scaled, seed_graph=GRAPH_SEED)
np.save("learned_adj_graph10k.npy", adj_matrix)
print("Graph structure learned and saved.")


Graph structure learned and saved.


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Hyperparameters
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SAGE_HIDDEN_DIM = 128
SAGE_DROPOUT = 0.2
LR = 4e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 15000
PATIENCE = 1000
SEEDS = [8]

class GraphSAGENet(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.out = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        x = F.relu(self.conv3(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        x = F.relu(self.conv4(x, edge_index))
        x = F.dropout(x, p=SAGE_DROPOUT, training=self.training)
        return self.out(x, edge_index)

# Load Data and Graph
df = pd.read_csv("10000_cylindrical_anomalies.csv")
X_raw = torch.tensor(df.iloc[:, :22].values, dtype=torch.float32)
Y_raw = torch.tensor(df.iloc[:, 22:23].values, dtype=torch.float32)
scaler = StandardScaler()
X = torch.tensor(scaler.fit_transform(np.abs(X_raw.numpy())), dtype=torch.float32)
adj = np.load("learned_adj_graph10k.npy")
edge_index = torch.tensor(np.vstack(adj.nonzero()), dtype=torch.long)

results_r2 = []
results_mse = []

for seed in SEEDS:
    np.random.seed(seed)
    torch.manual_seed(seed)
    if DEVICE.type == 'cuda':
        torch.cuda.manual_seed_all(seed)

    idx = np.arange(X.shape[0])
    train_idx, temp_idx = train_test_split(idx, test_size=0.2, random_state=seed)
    val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=seed)

    def get_mask(idxs):
        mask = torch.zeros(X.shape[0], dtype=torch.bool)
        mask[idxs] = True
        return mask

    data = Data(
        x=X.to(DEVICE),
        y=Y_raw.to(DEVICE),
        edge_index=edge_index.to(DEVICE)
    )
    masks = {k: get_mask(v).to(DEVICE) for k, v in zip(['train', 'val', 'test'], [train_idx, val_idx, test_idx])}

    model = GraphSAGENet(X.size(1), SAGE_HIDDEN_DIM, Y_raw.size(1)).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=200)
    criterion = torch.nn.MSELoss()

    best_val = np.inf
    patience_counter = 0

    for epoch in range(EPOCHS):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[masks['train']], data.y[masks['train']])
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_loss = criterion(model(data.x, data.edge_index)[masks['val']], data.y[masks['val']])
        scheduler.step(val_loss)

        if val_loss < best_val:
            best_val = val_loss
            best_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                break

    model.load_state_dict(best_state)
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        r2 = r2_score(data.y[masks['test']].cpu().numpy(), out[masks['test']].cpu().numpy())
        mse = mean_squared_error(data.y[masks['test']].cpu().numpy(), out[masks['test']].cpu().numpy())
        print(f"Seed {seed:3d} | R²: {r2:.4f} | MSE: {mse:.4f}")
        results_r2.append(r2)
        results_mse.append(mse)

print("\n--- R² Summary ---")
print(f"Mean R² = {np.mean(results_r2):.4f}, Std = {np.std(results_r2):.4f}")
print("\n--- MSE Summary ---")
print(f"Mean MSE = {np.mean(results_mse):.4f}, Std = {np.std(results_mse):.4f}")